In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# load dataset
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result

In [13]:
# cnn

def weight_variable(shape):
    # better distribution for random variable in cnn (google did that.)
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    # better to use positive value
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # strides [1, x_movement, y_movement, 1]
    # Must have strides[0] and strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    
def max_pooling_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


In [14]:
# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])
#print(x_image.shape) # [n_samples, 28, 28, 1]

In [15]:
## conv1 layer
W_conv1 = weight_variable([5, 5, 1, 32]) # patch: 5x5, in_size: 1, out_size: 32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # output size: 28x28x32
h_pool1 = max_pooling_2x2(h_conv1)                       # output size: 14x14x32

## conv2 layer
W_conv2 = weight_variable([5, 5, 32, 64]) # patch: 5x5, in_size: 32, out_size: 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size: 14x14x64
h_pool2 = max_pooling_2x2(h_conv2)                       # output size: 7 x7 x64

## func1 layer
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

# [n_sample, 7, 7, 64] ->> [n_sample, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) 
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=0.5)

## func2 layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [16]:
# error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_mean(ys*tf.log(prediction), reduction_indices=[1]))
# training
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [17]:
# initialize
sess = tf.Session()
sess.run(tf.initialize_all_variables())

In [ ]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(300)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
    if i % 50 == 0:
        print(compute_accuracy(mnist.test.images, mnist.test.labels))

0.1075
